# Synthea Data Cleaning

# Setup

## Libraries

In [1]:
import pandas as pd

## Load Tables

In [39]:
# Read in tables generated manually and by Synthea
## Manually 
man_claims_df = pd.read_csv('csv/claims_data.csv')
man_conditions_df = pd.read_csv('csv/conditions_data.csv')
man_medications_df = pd.read_csv('csv/medications_data.csv')
man_observations_df = pd.read_csv('csv/observations_data.csv')
man_procedures_df = pd.read_csv('csv/procedures_data.csv')
man_patients_df = pd.read_csv('csv/patient_data.csv')
## Synthea-generated
gen_claims_df = pd.read_csv('output/csv/claims.csv')
gen_conditions_df = pd.read_csv('output/csv/conditions.csv')
gen_medications_df = pd.read_csv('output/csv/medications.csv')
gen_observations_df = pd.read_csv('output/csv/observations.csv')
gen_procedures_df = pd.read_csv('output/csv/procedures.csv')
gen_patients_df = pd.read_csv('output/csv/patients.csv')
gen_encounters_df = pd.read_csv('output/csv/encounters.csv')
gen_devices_df = pd.read_csv('output/csv/devices.csv')
gen_organizations_df = pd.read_csv('output/csv/organizations.csv')
gen_payers_df = pd.read_csv('output/csv/payers.csv')
gen_providers_df = pd.read_csv('output/csv/providers.csv')

# Compare Tables

In [41]:
# Compare number of entries between tables
print('# of Patients in Manual Table:', man_patients_df.shape)
print('# of Patients in Synthea-generated Table:', gen_patients_df.shape)

# of Patients in Manual Table: (10471, 14)
# of Patients in Synthea-generated Table: (10471, 27)


In [46]:
print('Manual Columns:', man_patients_df.columns)
print('Synthea Columns:', gen_patients_df.columns)

Manual Columns: Index(['uuid', 'first_name', 'middle_name', 'family_name', 'gender',
       'birth_date', 'marital_status', 'general_practioner', 'age_years',
       'country', 'state', 'city', 'location_lat', 'location_long'],
      dtype='object')
Synthea Columns: Index(['Id', 'BIRTHDATE', 'DEATHDATE', 'SSN', 'DRIVERS', 'PASSPORT', 'PREFIX',
       'FIRST', 'LAST', 'SUFFIX', 'MAIDEN', 'MARITAL', 'RACE', 'ETHNICITY',
       'GENDER', 'BIRTHPLACE', 'ADDRESS', 'CITY', 'STATE', 'COUNTY', 'FIPS',
       'ZIP', 'LAT', 'LON', 'HEALTHCARE_EXPENSES', 'HEALTHCARE_COVERAGE',
       'INCOME'],
      dtype='object')


Seems like there are some columns that we were not able to find manually in the FIHR file, let's use the CSV file that Synthea generated. Still was a good exercise to better understand the FIHR file format!

# Cleaning

## Claims

In [64]:
gen_claims_df.describe(include='O')

,Id,PATIENTID,PROVIDERID,PRIMARYPATIENTINSURANCEID,SECONDARYPATIENTINSURANCEID,APPOINTMENTID,CURRENTILLNESSDATE,SERVICEDATE,SUPERVISINGPROVIDERID,STATUS1,STATUS2,STATUSP,LASTBILLEDDATE1,LASTBILLEDDATE2,LASTBILLEDDATEP
count,1301040,1301040,1301040,1191264,372646,1301040,1301040,1301040,1301040,1301040,928351,1301040,1301040,928351,1301040
unique,1301040,10471,4603,10,2,663881,521528,655268,4603,2,2,2,662708,510586,662708
top,8d943498-10b8-ba65-76f3-0cfcb056b120,54bdf4be-f9a4-3573-b2ba-e635149f12f1,67db2e79-591c-3c8e-b376-82c8a40923cc,a735bf55-83e9-331a-899d-a82a60b9f60c,b046940f-1664-3047-bca7-dfa76be352a4,4a837da4-bac6-c55d-564c-bef7728d7145,2018-11-29T21:51:17Z,2020-12-30T18:40:05Z,67db2e79-591c-3c8e-b376-82c8a40923cc,CLOSED,CLOSED,CLOSED,2021-01-10T23:01:10Z,2020-12-11T23:34:35Z,2021-01-10T23:01:10Z
freq,1,3862,24261,472628,372634,85,1162,85,24261,1299735,927615,1299735,85,64,85


In [65]:
gen_claims_df.describe()

,DEPARTMENTID,PATIENTDEPARTMENTID,DIAGNOSIS1,DIAGNOSIS2,DIAGNOSIS3,DIAGNOSIS4,DIAGNOSIS5,DIAGNOSIS6,DIAGNOSIS7,DIAGNOSIS8,REFERRINGPROVIDERID,OUTSTANDING1,OUTSTANDING2,OUTSTANDINGP,HEALTHCARECLAIMTYPEID1,HEALTHCARECLAIMTYPEID2
count,1.301040e+06,1.301040e+06,1.301040e+06,3.155850e+05,1.009380e+05,2.735500e+04,9.121000e+03,3.457000e+03,1.263000e+03,2.360000e+02,0.0,1.301040e+06,928351.000000,1.301040e+06,1.301040e+06,1.301040e+06
mean,5.186977e+00,5.186977e+00,8.688700e+12,6.546000e+13,1.841365e+14,2.669693e+14,2.016812e+14,1.625597e+14,1.475857e+14,4.552818e+08,NaN,4.212866e-01,0.323312,1.607995e-01,1.551757e+00,1.119219e+00
std,6.757684e+00,6.757684e+00,2.953639e+14,8.336029e+14,1.398577e+15,1.686045e+15,1.421757e+15,1.252514e+15,1.102578e+15,3.348527e+08,NaN,5.517990e+01,48.662847,2.765947e+01,4.973142e-01,8.233565e-01
min,1.000000e+00,1.000000e+00,1.734006e+06,4.557003e+06,1.734006e+06,1.577700e+07,1.577700e+07,1.577700e+07,3.291100e+07,3.695501e+07,NaN,0.000000e+00,0.000000,0.000000e+00,1.000000e+00,0.000000e+00
25%,1.000000e+00,1.000000e+00,9.426000e+07,7.359500e+07,7.359500e+07,7.359500e+07,6.778200e+07,3.866610e+08,3.866610e+08,7.359500e+07,NaN,0.000000e+00,0.000000,0.000000e+00,1.000000e+00,0.000000e+00
50%,2.000000e+00,2.000000e+00,3.145290e+08,1.609040e+08,2.788600e+08,2.718250e+08,3.610550e+08,7.068700e+08,7.068700e+08,3.866610e+08,NaN,0.000000e+00,0.000000,0.000000e+00,2.000000e+00,1.000000e+00
75%,5.000000e+00,5.000000e+00,4.318570e+08,4.226500e+08,4.233150e+08,4.243930e+08,7.068930e+08,7.703490e+08,1.322810e+14,8.405390e+08,NaN,0.000000e+00,0.000000,0.000000e+00,2.000000e+00,2.000000e+00
max,2.000000e+01,2.000000e+01,1.093988e+16,1.093988e+16,1.093988e+16,1.093988e+16,1.093988e+16,1.093988e+16,1.093988e+16,8.405390e+08,NaN,1.578483e+04,15784.830000,1.578483e+04,2.000000e+00,2.000000e+00


In [97]:
cols_io_fix = ['DEPARTMENTID', 'PATIENTDEPARTMENTID', 'DIAGNOSIS1', 'DIAGNOSIS2', 'DIAGNOSIS3', 'DIAGNOSIS4', 'DIAGNOSIS5', 'DIAGNOSIS6', 'DIAGNOSIS7', 'DIAGNOSIS8', 'REFERRINGPROVIDERID', 'HEALTHCARECLAIMTYPEID1', 'HEALTHCARECLAIMTYPEID2']
cols_od_fix = ['CURRENTILLNESSDATE', 'SERVICEDATE', 'LASTBILLEDDATE1', 'LASTBILLEDDATE2', 'LASTBILLEDDATEP']

gen_claims_cleaned = gen_claims_df.copy()

gen_claims_cleaned[cols_io_fix] = gen_claims_cleaned[cols_io_fix].astype('O')
for col in cols_od_fix:
    gen_claims_cleaned[col] = pd.to_datetime(gen_claims_cleaned[col], format='%Y-%m-%dT%H:%M:%SZ')

gen_claims_cleaned.dtypes

Id                                     object
PATIENTID                              object
PROVIDERID                             object
PRIMARYPATIENTINSURANCEID              object
SECONDARYPATIENTINSURANCEID            object
DEPARTMENTID                           object
PATIENTDEPARTMENTID                    object
DIAGNOSIS1                             object
DIAGNOSIS2                             object
DIAGNOSIS3                             object
DIAGNOSIS4                             object
DIAGNOSIS5                             object
DIAGNOSIS6                             object
DIAGNOSIS7                             object
DIAGNOSIS8                             object
REFERRINGPROVIDERID                    object
APPOINTMENTID                          object
CURRENTILLNESSDATE             datetime64[ns]
SERVICEDATE                    datetime64[ns]
SUPERVISINGPROVIDERID                  object
STATUS1                                object
STATUS2                           

## Conditions

In [98]:
gen_conditions_df.describe(include='O')

,START,STOP,PATIENT,ENCOUNTER,DESCRIPTION
count,352528,255574,352528,352528,352528
unique,22559,16449,10471,230697,288
top,2014-01-31,2023-04-08,85ff2938-f822-2563-8066-cac645f9f673,4917e0f2-ed1a-d73f-caca-3d9b7a6351dc,Medication review due (situation)
freq,659,171,513,12,84687


In [99]:
gen_conditions_df.describe()

,CODE
count,3.525280e+05
mean,6.272974e+13
std,8.178066e+14
min,1.734006e+06
25%,1.256050e+08
50%,2.745310e+08
75%,3.145290e+08
max,1.093988e+16


In [100]:
gen_conditions_cleaned = gen_conditions_df.copy()
gen_conditions_cleaned['CODE'] = gen_conditions_cleaned['CODE'].astype('O')

## Devices

In [108]:
gen_devices_df.head(1)

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION,UDI
0,2013-05-01T07:37:50Z,NaN,66d620df-29b2-b3b7-cd7e-1cb6db4ecf33,4c79844e-2377-c70f-6e37-dc38cf5d1a48,170615005,Home nebulizer (physical object),(01)43039098460415(11)130410(17)380425(10)1049...


In [109]:
gen_devices_df.describe(include='O')

,START,STOP,PATIENT,ENCOUNTER,DESCRIPTION,UDI
count,16320,7654,16320,16320,16320,16320
unique,12844,5153,6166,12548,29,16320
top,2010-01-20T02:27:55Z,2003-03-22T11:18:06Z,593f438c-13f4-8662-2820-ac9f593b1a79,71787c9e-2a83-1d01-0809-33279f633364,Blood glucose meter (physical object),(01)43039098460415(11)130410(17)380425(10)1049...
freq,5,4,94,11,4036,1


In [110]:
gen_devices_df.describe()

,CODE
count,1.632000e+04
mean,4.176817e+08
std,2.247549e+08
min,1.410601e+07
25%,2.288690e+08
50%,3.374140e+08
75%,7.021720e+08
max,8.605770e+08


In [112]:
cols_io_fix = ['CODE']
cols_od_fix = ['START', 'STOP']

gen_devices_cleaned = gen_devices_df.copy()

gen_devices_cleaned[cols_io_fix] = gen_devices_cleaned[cols_io_fix].astype('O')
for col in cols_od_fix:
    gen_devices_cleaned[col] = pd.to_datetime(gen_devices_cleaned[col], format='%Y-%m-%dT%H:%M:%SZ')

gen_devices_cleaned.dtypes

START          datetime64[ns]
STOP           datetime64[ns]
PATIENT                object
ENCOUNTER              object
CODE                   object
DESCRIPTION            object
UDI                    object
dtype: object

## Encounters

In [105]:
gen_encounters_df.head(1)

,Id,START,STOP,PATIENT,ORGANIZATION,PROVIDER,PAYER,ENCOUNTERCLASS,CODE,DESCRIPTION,BASE_ENCOUNTER_COST,TOTAL_CLAIM_COST,PAYER_COVERAGE,REASONCODE,REASONDESCRIPTION
0,a21539cd-9371-86cb-3b1d-1a5d8dd8e1ce,1997-06-01T03:37:50Z,1997-06-01T03:52:50Z,66d620df-29b2-b3b7-cd7e-1cb6db4ecf33,06630d14-645e-3403-b94c-e3ed710ca998,85e06377-c0d0-3f31-a860-83867985c68f,df166300-5a78-3502-a46a-832842197811,ambulatory,185347001,Encounter for problem,82.02,82.02,32.02,735029006.0,Shellfish (substance)


In [103]:
gen_encounters_df.describe(include='O')

,Id,START,STOP,PATIENT,ORGANIZATION,PROVIDER,PAYER,ENCOUNTERCLASS,DESCRIPTION,REASONDESCRIPTION
count,663881,663881,663881,663881,663881,663881,663881,663881,663881,416486
unique,663881,655268,662692,10471,4603,4603,10,10,70,162
top,a21539cd-9371-86cb-3b1d-1a5d8dd8e1ce,1941-04-14T17:07:02Z,2023-12-28T22:50:19Z,9b28eee9-1007-e14d-8ba3-92d29873f3e6,17260c93-fcaf-3ccf-815b-0ddb786f5f6d,67db2e79-591c-3c8e-b376-82c8a40923cc,a735bf55-83e9-331a-899d-a82a60b9f60c,ambulatory,Encounter for problem (procedure),Chronic kidney disease stage 4 (disorder)
freq,1,11,12,822,13191,13191,192866,368460,231995,129669


In [104]:
gen_encounters_df.describe()

,CODE,BASE_ENCOUNTER_COST,TOTAL_CLAIM_COST,PAYER_COVERAGE,REASONCODE
count,6.638810e+05,663881.000000,6.638810e+05,6.638810e+05,4.164860e+05
mean,1.062157e+12,111.322763,3.813471e+03,2.901775e+03,5.804521e+11
std,2.182452e+13,31.710628,1.538190e+04,1.132638e+04,1.036281e+13
min,1.505002e+06,75.000000,0.000000e+00,0.000000e+00,1.734006e+06
25%,1.853450e+08,82.020000,6.997300e+02,6.562000e+01,5.582200e+07
50%,1.853470e+08,82.020000,1.116230e+03,7.954100e+02,2.745310e+08
75%,3.083350e+08,136.700000,1.685230e+03,1.335560e+03,4.318570e+08
max,4.531310e+14,183.230000,2.019367e+06,1.482917e+06,4.425710e+14


In [106]:
cols_io_fix = ['CODE', 'REASONCODE']

gen_encounters_cleaned = gen_encounters_df.copy()

gen_encounters_cleaned[cols_io_fix] = gen_encounters_cleaned[cols_io_fix].astype('O')
gen_encounters_cleaned.dtypes

Id                      object
START                   object
STOP                    object
PATIENT                 object
ORGANIZATION            object
PROVIDER                object
PAYER                   object
ENCOUNTERCLASS          object
CODE                    object
DESCRIPTION             object
BASE_ENCOUNTER_COST    float64
TOTAL_CLAIM_COST       float64
PAYER_COVERAGE         float64
REASONCODE              object
REASONDESCRIPTION       object
dtype: object

## Medications

In [113]:
gen_medications_df.head(1)

,START,STOP,PATIENT,PAYER,ENCOUNTER,CODE,DESCRIPTION,BASE_COST,PAYER_COVERAGE,DISPENSES,TOTALCOST,REASONCODE,REASONDESCRIPTION
0,1997-06-13T09:01:20Z,NaN,66d620df-29b2-b3b7-cd7e-1cb6db4ecf33,df166300-5a78-3502-a46a-832842197811,1252981e-81f8-8c17-b55a-329c92478049,997488,Fexofenadine hydrochloride 30 MG Oral Tablet,752.51,702.51,323,243060.73,NaN,NaN


In [114]:
gen_medications_df.describe(include='O')

,START,STOP,PATIENT,PAYER,ENCOUNTER,DESCRIPTION,REASONDESCRIPTION
count,637159,609053,637159,637159,637159,637159,547379
unique,363351,347315,9974,10,341600,347,58
top,2011-08-10T02:27:55Z,2011-08-10T02:27:55Z,54bdf4be-f9a4-3573-b2ba-e635149f12f1,a735bf55-83e9-331a-899d-a82a60b9f60c,4a837da4-bac6-c55d-564c-bef7728d7145,1 ML Epoetin Alfa 4000 UNT/ML Injection [Epogen],Essential hypertension (disorder)
freq,16,15,3384,279748,84,161508,179839


In [115]:
gen_medications_df.describe()

,CODE,BASE_COST,PAYER_COVERAGE,DISPENSES,TOTALCOST,REASONCODE
count,6.371590e+05,637159.000000,637159.000000,637159.000000,6.371590e+05,5.473790e+05
mean,4.617809e+05,236.709681,161.794618,5.563897,5.912087e+03,8.570849e+11
std,4.389704e+05,2400.491936,1892.796794,35.188444,4.054988e+05,7.915607e+12
min,1.050780e+05,0.010000,0.000000,1.000000,1.000000e-02,5.602001e+06
25%,2.059230e+05,1.990000,0.000000,1.000000,4.150000e+00,5.962100e+07
50%,3.103250e+05,59.890000,11.620000,1.000000,5.997000e+01,5.962100e+07
75%,5.832140e+05,260.780000,142.995000,2.000000,4.031950e+02,2.717370e+08
max,2.563431e+06,207412.800000,207377.800000,1645.000000,8.157134e+07,1.322810e+14


In [116]:
cols_io_fix = ['CODE', 'REASONCODE']
cols_od_fix = ['START', 'STOP']

gen_medications_cleaned = gen_medications_df.copy()

gen_medications_cleaned[cols_io_fix] = gen_medications_cleaned[cols_io_fix].astype('O')
for col in cols_od_fix:
    gen_medications_cleaned[col] = pd.to_datetime(gen_medications_cleaned[col], format='%Y-%m-%dT%H:%M:%SZ')

gen_medications_cleaned.dtypes

START                datetime64[ns]
STOP                 datetime64[ns]
PATIENT                      object
PAYER                        object
ENCOUNTER                    object
CODE                         object
DESCRIPTION                  object
BASE_COST                   float64
PAYER_COVERAGE              float64
DISPENSES                     int64
TOTALCOST                   float64
REASONCODE                   object
REASONDESCRIPTION            object
dtype: object

## Observations

In [117]:
gen_observations_df.head(1)

,DATE,PATIENT,ENCOUNTER,CATEGORY,CODE,DESCRIPTION,VALUE,UNITS,TYPE
0,2014-10-23T01:37:50Z,66d620df-29b2-b3b7-cd7e-1cb6db4ecf33,b25682c5-b428-4e16-2fff-90064618cc56,vital-signs,8302-2,Body Height,166.5,cm,numeric


In [118]:
gen_observations_df.describe(include='O')

,DATE,PATIENT,ENCOUNTER,CATEGORY,CODE,DESCRIPTION,VALUE,UNITS,TYPE
count,9349291,9349291,9057454,9057454,9349291,9349291,9349291,6931549,9349291
unique,947405,10471,442681,8,293,310,23735,49,2
top,1964-09-19T09:25:25Z,85ff2938-f822-2563-8066-cac645f9f673,31cd4094-6648-5c92-3ed9-df9f6524f01d,laboratory,72514-3,Pain severity - 0-10 verbal numeric rating [Sc...,No,mg/dL,numeric
freq,184,25935,1483,4348136,353474,353474,817679,1383655,5751263


In [121]:
gen_observations_df['VALUE'][gen_observations_df['VALUE'].str.contains(r'^[A-Z].*.')]

9                   Never smoked tobacco (finding)
10                                              No
11                                             Yes
12                                              No
13                                              No
                            ...                   
9349245          Urine protein test = ++ (finding)
9349246           Urine nitrite negative (finding)
9349247      Urine blood test = negative (finding)
9349248    Urine leukocyte test negative (finding)
9349257                        Ex-smoker (finding)
Name: VALUE, Length: 3258087, dtype: object

In [122]:
gen_observations_df.describe()

,DATE,PATIENT,ENCOUNTER,CATEGORY,CODE,DESCRIPTION,VALUE,UNITS,TYPE
count,9349291,9349291,9057454,9057454,9349291,9349291,9349291,6931549,9349291
unique,947405,10471,442681,8,293,310,23735,49,2
top,1964-09-19T09:25:25Z,85ff2938-f822-2563-8066-cac645f9f673,31cd4094-6648-5c92-3ed9-df9f6524f01d,laboratory,72514-3,Pain severity - 0-10 verbal numeric rating [Sc...,No,mg/dL,numeric
freq,184,25935,1483,4348136,353474,353474,817679,1383655,5751263


In [124]:
cols_od_fix = ['DATE']

gen_observations_cleaned = gen_observations_df.copy()

for col in cols_od_fix:
    gen_observations_cleaned[col] = pd.to_datetime(gen_observations_cleaned[col], format='%Y-%m-%dT%H:%M:%SZ')

gen_observations_cleaned.dtypes

DATE           datetime64[ns]
PATIENT                object
ENCOUNTER              object
CATEGORY               object
CODE                   object
DESCRIPTION            object
VALUE                  object
UNITS                  object
TYPE                   object
dtype: object

## Organizations

In [126]:
gen_organizations_df.head(1)

,Id,NAME,ADDRESS,CITY,STATE,ZIP,LAT,LON,PHONE,REVENUE,UTILIZATION
0,17260c93-fcaf-3ccf-815b-0ddb786f5f6d,HOLLYWOOD CROSS MEDICAL CLINIC,1110 N WESTERN AVE,LOS ANGELES,CA,900291087,34.053691,-118.242766,3234636881,0.0,17272


In [127]:
gen_organizations_df.describe(include='O')

,Id,NAME,ADDRESS,CITY,STATE,PHONE
count,4908,4908,4908,4908,4908,4908
unique,4908,4096,4722,695,1,4496
top,17260c93-fcaf-3ccf-815b-0ddb786f5f6d,FAMILY HEALTH CENTERS OF SAN DIEGO INC,16237 VENTURA BLVD,LOS ANGELES,CA,4153919686
freq,1,21,4,309,4908,10


In [128]:
gen_organizations_df.describe()

,ZIP,LAT,LON,REVENUE,UTILIZATION
count,4.908000e+03,4908.000000,4908.000000,4908.0,4908.000000
mean,8.876706e+08,35.429984,-119.314822,0.0,354.769967
std,1.904113e+08,2.090924,1.918651,0.0,837.977168
min,9.000300e+04,32.558661,-124.201736,0.0,1.000000
25%,9.131649e+08,33.924720,-121.291111,0.0,17.000000
50%,9.254371e+08,34.179267,-118.405637,0.0,79.000000
75%,9.450145e+08,37.500430,-117.935759,0.0,274.000000
max,9.863137e+08,41.928168,-114.359131,0.0,17272.000000


In [130]:
gen_organizations_cleaned = gen_organizations_df.copy()
gen_organizations_cleaned['ZIP'] = gen_organizations_cleaned['ZIP'].astype('O')

gen_organizations_cleaned.dtypes

Id              object
NAME            object
ADDRESS         object
CITY            object
STATE           object
ZIP             object
LAT            float64
LON            float64
PHONE           object
REVENUE        float64
UTILIZATION      int64
dtype: object

## Patients

In [131]:
gen_patients_df.head(1)

,Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,...,CITY,STATE,COUNTY,FIPS,ZIP,LAT,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE,INCOME
0,66d620df-29b2-b3b7-cd7e-1cb6db4ecf33,1996-08-28,NaN,999-42-8950,S99944559,X67939432X,Mr.,Clemente531,Botsford977,NaN,...,Jurupa Valley,California,Riverside County,6065.0,92509,34.009116,-117.429424,154180.29,994594.22,1462


In [132]:
gen_patients_df.describe(include='O')

,Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,MAIDEN,MARITAL,RACE,ETHNICITY,GENDER,BIRTHPLACE,ADDRESS,CITY,STATE,COUNTY
count,10471,10471,1335,10471,8464,7971,8223,10471,10471,87,2673,6802,10471,10471,10471,10471,10471,10471,10471,10471
unique,10471,7925,1293,10412,8088,7971,3,3827,930,3,706,4,6,2,2,884,10471,831,1,56
top,66d620df-29b2-b3b7-cd7e-1cb6db4ecf33,1913-02-08,2023-01-17,999-86-2200,S99915300,X67939432X,Mr.,Gilberto712,Yost751,MD,Smitham825,M,white,nonhispanic,M,Los Angeles California US,776 Littel Meadow Suite 28,Los Angeles,California,Los Angeles County
freq,1,11,3,2,4,1,4129,19,35,30,14,4083,7539,7065,5241,989,1,1079,10471,2765


In [133]:
gen_patients_df.describe()

,FIPS,ZIP,LAT,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE,INCOME
count,9937.000000,10471.000000,10471.000000,10471.000000,1.047100e+04,1.047100e+04,10471.000000
mean,6055.612458,88196.135804,35.432477,-119.377985,2.116675e+05,5.174888e+05,110317.957884
std,26.403587,20517.156364,2.073499,1.983411,3.503115e+05,7.328485e+05,164459.951541
min,6001.000000,0.000000,32.519445,-124.217383,1.000000e+02,0.000000e+00,17.000000
25%,6037.000000,91371.000000,33.878920,-121.540015,1.554231e+04,2.388458e+04,30412.000000
50%,6059.000000,92624.000000,34.183219,-118.396012,6.975178e+04,2.045489e+05,62826.000000
75%,6073.000000,94524.000000,37.583321,-117.846053,2.313118e+05,8.130585e+05,115814.500000
max,6115.000000,96161.000000,41.845624,-114.311945,8.096738e+06,1.067229e+07,998063.000000


In [135]:
cols_io_fix = ['ZIP']
cols_od_fix = ['BIRTHDATE', 'DEATHDATE']

gen_patients_cleaned = gen_patients_df.copy()

gen_patients_cleaned[cols_io_fix] = gen_patients_cleaned[cols_io_fix].astype('O')
for col in cols_od_fix:
    gen_patients_cleaned[col] = pd.to_datetime(gen_patients_cleaned[col], format='%Y-%m-%d')

gen_patients_cleaned.dtypes

Id                             object
BIRTHDATE              datetime64[ns]
DEATHDATE              datetime64[ns]
SSN                            object
DRIVERS                        object
PASSPORT                       object
PREFIX                         object
FIRST                          object
LAST                           object
SUFFIX                         object
MAIDEN                         object
MARITAL                        object
RACE                           object
ETHNICITY                      object
GENDER                         object
BIRTHPLACE                     object
ADDRESS                        object
CITY                           object
STATE                          object
COUNTY                         object
FIPS                          float64
ZIP                            object
LAT                           float64
LON                           float64
HEALTHCARE_EXPENSES           float64
HEALTHCARE_COVERAGE           float64
INCOME      

## Payers

In [137]:
gen_payers_df.head(1)

,Id,NAME,OWNERSHIP,ADDRESS,CITY,STATE_HEADQUARTERED,ZIP,PHONE,AMOUNT_COVERED,AMOUNT_UNCOVERED,...,UNCOVERED_ENCOUNTERS,COVERED_MEDICATIONS,UNCOVERED_MEDICATIONS,COVERED_PROCEDURES,UNCOVERED_PROCEDURES,COVERED_IMMUNIZATIONS,UNCOVERED_IMMUNIZATIONS,UNIQUE_CUSTOMERS,QOLS_AVG,MEMBER_MONTHS
0,a735bf55-83e9-331a-899d-a82a60b9f60c,Medicare,GOVERNMENT,NaN,NaN,NaN,NaN,NaN,1.026231e+09,41679017.13,...,0,415899,0,455535,0,50457,0,3188,0.657503,513864


In [139]:
gen_payers_df.describe(include='O')

,Id,NAME,OWNERSHIP
count,10,10,10
unique,10,10,3
top,a735bf55-83e9-331a-899d-a82a60b9f60c,Medicare,PRIVATE
freq,1,1,6


In [140]:
gen_payers_df.describe()

,ADDRESS,CITY,STATE_HEADQUARTERED,ZIP,PHONE,AMOUNT_COVERED,AMOUNT_UNCOVERED,REVENUE,COVERED_ENCOUNTERS,UNCOVERED_ENCOUNTERS,COVERED_MEDICATIONS,UNCOVERED_MEDICATIONS,COVERED_PROCEDURES,UNCOVERED_PROCEDURES,COVERED_IMMUNIZATIONS,UNCOVERED_IMMUNIZATIONS,UNIQUE_CUSTOMERS,QOLS_AVG,MEMBER_MONTHS
count,0.0,0.0,0.0,0.0,0.0,1.000000e+01,1.000000e+01,1.000000e+01,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,1.000000e+01
mean,NaN,NaN,NaN,NaN,NaN,5.844089e+08,2.403922e+08,8.165658e+07,139244.400000,34876.600000,106098.700000,17899.200000,196855.500000,50700.200000,44564.200000,14456.100000,2748.100000,0.782100,5.860572e+05
std,NaN,NaN,NaN,NaN,NaN,6.620388e+08,5.090483e+08,9.618183e+07,141743.143117,110289.493043,123438.345175,56602.240295,197352.054398,160328.109826,50485.759893,45714.202083,1533.477417,0.164500,5.147253e+05
min,NaN,NaN,NaN,NaN,NaN,0.000000e+00,3.682046e+06,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,952.000000,0.415782,1.086600e+05
25%,NaN,NaN,NaN,NaN,NaN,1.889587e+08,3.809667e+07,1.669606e+07,60725.250000,0.000000,46128.500000,0.000000,85653.750000,0.000000,16211.250000,0.000000,1956.000000,0.694448,1.911990e+05
50%,NaN,NaN,NaN,NaN,NaN,3.155267e+08,6.790325e+07,3.307612e+07,72188.000000,0.000000,53398.500000,0.000000,104385.000000,0.000000,26366.500000,0.000000,2151.500000,0.813841,3.993420e+05
75%,NaN,NaN,NaN,NaN,NaN,7.822280e+08,1.001947e+08,1.268171e+08,198988.250000,0.000000,119525.750000,0.000000,284284.250000,0.000000,47866.750000,0.000000,3524.000000,0.900721,7.944390e+05
max,NaN,NaN,NaN,NaN,NaN,2.228180e+09,1.671142e+09,2.760412e+08,438721.000000,348766.000000,415899.000000,178992.000000,599584.000000,507002.000000,169984.000000,144561.000000,5822.000000,0.949791,1.473432e+06


In [141]:
gen_payers_cleaned = gen_payers_df.copy()
gen_payers_cleaned = gen_payers_cleaned.drop(columns=['ADDRESS', 'CITY', 'STATE_HEADQUARTERED', 'ZIP', 'PHONE'])

gen_payers_cleaned.dtypes

Id                          object
NAME                        object
OWNERSHIP                   object
AMOUNT_COVERED             float64
AMOUNT_UNCOVERED           float64
REVENUE                    float64
COVERED_ENCOUNTERS           int64
UNCOVERED_ENCOUNTERS         int64
COVERED_MEDICATIONS          int64
UNCOVERED_MEDICATIONS        int64
COVERED_PROCEDURES           int64
UNCOVERED_PROCEDURES         int64
COVERED_IMMUNIZATIONS        int64
UNCOVERED_IMMUNIZATIONS      int64
UNIQUE_CUSTOMERS             int64
QOLS_AVG                   float64
MEMBER_MONTHS                int64
dtype: object

## Procedures

In [142]:
gen_procedures_df.head(1)

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION,BASE_COST,REASONCODE,REASONDESCRIPTION
0,2014-10-23T01:37:50Z,2014-10-23T01:52:50Z,66d620df-29b2-b3b7-cd7e-1cb6db4ecf33,b25682c5-b428-4e16-2fff-90064618cc56,430193006,Medication Reconciliation (procedure),713.31,NaN,NaN


In [143]:
gen_procedures_df.describe(include='O')

,START,STOP,PATIENT,ENCOUNTER,DESCRIPTION,REASONDESCRIPTION
count,1296502,1296502,1296502,1296502,1296502,402635
unique,1069167,1144940,10449,475786,364,90
top,1965-02-28T22:42:23Z,2017-05-31T06:35:32Z,69c753ca-9561-0ec0-6991-1a0042a7c787,405b9054-fd33-cb9d-dd62-8f06dd5ad133,Renal dialysis (procedure),Normal pregnancy
freq,25,25,1699,174,185098,137648


In [144]:
gen_procedures_df.describe()

,CODE,BASE_COST,REASONCODE
count,1.296502e+06,1.296502e+06,4.026350e+05
mean,8.510712e+13,1.882297e+03,5.383977e+11
std,6.855398e+14,5.910516e+03,6.079740e+12
min,1.660010e+05,5.900000e-01,1.734006e+06
25%,2.657640e+08,6.005000e+02,7.289200e+07
50%,4.104010e+08,6.005000e+02,7.289200e+07
75%,7.114460e+08,9.988500e+02,4.318570e+08
max,1.633503e+16,9.218699e+05,3.685810e+14


In [145]:
cols_io_fix = ['CODE', 'REASONCODE']
cols_od_fix = ['START', 'STOP']

gen_procedures_cleaned = gen_procedures_df.copy()

gen_procedures_cleaned[cols_io_fix] = gen_procedures_cleaned[cols_io_fix].astype('O')
for col in cols_od_fix:
    gen_procedures_cleaned[col] = pd.to_datetime(gen_procedures_cleaned[col], format='%Y-%m-%dT%H:%M:%SZ')

gen_procedures_cleaned.dtypes

START                datetime64[ns]
STOP                 datetime64[ns]
PATIENT                      object
ENCOUNTER                    object
CODE                         object
DESCRIPTION                  object
BASE_COST                   float64
REASONCODE                   object
REASONDESCRIPTION            object
dtype: object

## Providers

In [147]:
gen_providers_df.head(1)

,Id,ORGANIZATION,NAME,GENDER,SPECIALITY,ADDRESS,CITY,STATE,ZIP,LAT,LON,ENCOUNTERS,PROCEDURES
0,67db2e79-591c-3c8e-b376-82c8a40923cc,17260c93-fcaf-3ccf-815b-0ddb786f5f6d,Alise864 Jast432,F,GENERAL PRACTICE,1110 N WESTERN AVE,LOS ANGELES,CA,900291087,34.053691,-118.242766,17272,0


In [148]:
gen_providers_df.describe(include='O')

,Id,ORGANIZATION,NAME,GENDER,SPECIALITY,ADDRESS,CITY,STATE
count,4908,4908,4908,4908,4908,4908,4908,4908
unique,4908,4908,4891,2,1,4722,695,1
top,67db2e79-591c-3c8e-b376-82c8a40923cc,17260c93-fcaf-3ccf-815b-0ddb786f5f6d,Bertram873 Kutch271,F,GENERAL PRACTICE,16237 VENTURA BLVD,LOS ANGELES,CA
freq,1,1,2,2533,4908,4,309,4908


In [149]:
gen_providers_df.describe()

,ZIP,LAT,LON,ENCOUNTERS,PROCEDURES
count,4.908000e+03,4908.000000,4908.000000,4908.000000,4908.0
mean,8.876706e+08,35.429984,-119.314822,354.769967,0.0
std,1.904113e+08,2.090924,1.918651,837.977168,0.0
min,9.000300e+04,32.558661,-124.201736,1.000000,0.0
25%,9.131649e+08,33.924720,-121.291111,17.000000,0.0
50%,9.254371e+08,34.179267,-118.405637,79.000000,0.0
75%,9.450145e+08,37.500430,-117.935759,274.000000,0.0
max,9.863137e+08,41.928168,-114.359131,17272.000000,0.0


In [150]:
gen_providers_cleaned = gen_providers_df.copy()
gen_providers_cleaned['ZIP'] = gen_providers_cleaned['ZIP'].astype('O')

gen_providers_cleaned.dtypes

Id               object
ORGANIZATION     object
NAME             object
GENDER           object
SPECIALITY       object
ADDRESS          object
CITY             object
STATE            object
ZIP              object
LAT             float64
LON             float64
ENCOUNTERS        int64
PROCEDURES        int64
dtype: object

# Output

In [151]:
gen_claims_cleaned.to_csv('data_cleaned/claims_cleaned.csv', index=False)
gen_conditions_cleaned.to_csv('data_cleaned/conditions_cleaned.csv', index=False)
gen_medications_cleaned.to_csv('data_cleaned/medications_cleaned.csv', index=False)
gen_observations_cleaned.to_csv('data_cleaned/observations_cleaned.csv', index=False)
gen_procedures_cleaned.to_csv('data_cleaned/procedures_cleaned.csv', index=False)
gen_patients_cleaned.to_csv('data_cleaned/patients_cleaned.csv', index=False)
gen_encounters_cleaned.to_csv('data_cleaned/encounters_cleaned.csv', index=False)
gen_devices_cleaned.to_csv('data_cleaned/devices_cleaned.csv', index=False)
gen_organizations_cleaned.to_csv('data_cleaned/organizations_cleaned.csv', index=False)
gen_payers_cleaned.to_csv('data_cleaned/payers_cleaned.csv', index=False)
gen_providers_cleaned.to_csv('data_cleaned/providers_cleaned.csv', index=False)